# Lab 04: Linear Regression and Causality
cross-sectional estimator

This lab asks you to do two tasks:

First, to estimate the effect of Right Hearth
Catheterization (RHC)--inserting a tiny hollow tube along the arterial into
the right side of your hearth.  It is often done for diagnostical
purposes (it allows to get different measures right in the hearth) and
it's usually considered safe.

Wou will use a dataset about RHC for critically ill patients and see
if RHC is related to increased death rate.  The dataset is downloaded
from [Vanderbilt
Biostats](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) and
more information is available at
[http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html).

Second, you have to create (or re-create) the design matrices of the
first problem, analyze their properties, and compute the regression
coefficients of the first part "manually", i.e. by a simple matrix
expression on computer.

You can use both R and python.

In [12]:
## import the necessary tools, and check their corresponding versions
import numpy as np
print("numpy:", np.__version__)
import pandas as pd
print("pandas:", pd.__version__)
import matplotlib.pyplot as plt
import codecs
import scipy

import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

numpy: 1.12.1
pandas: 0.20.1


## 1 Get Ready

Load the data.  A tab-separated version is available on canvas (files/data).

1. How many observation and variables do we have?

Observation: 5735
Variables: 62

2. The most important variables we use below are _death_ (patient
death recorded/not recorded) and
_swang1_ (rhc performed/not performed).  How are these variables coded?

They are coded in string

In [13]:
print("1. How many observation and variables do we have?")
df1 = pd.read_csv('hearth_catheterization.csv', sep = "\t")
print(df1)
print("Variables: ", len(df1.columns))
print("Observation: ", len(df1.index))

1. How many observation and variables do we have?
                   cat1               cat2          ca  sadmdte  dschdte  \
0                  COPD                NaN         Yes    11142  11151.0   
1         MOSF w/Sepsis                NaN          No    11799  11844.0   
2     MOSF w/Malignancy      MOSF w/Sepsis         Yes    12083  12143.0   
3                   ARF                NaN          No    11146  11183.0   
4         MOSF w/Sepsis                NaN          No    12035  12037.0   
5                  COPD                NaN          No    12389  12396.0   
6     MOSF w/Malignancy                NaN  Metastatic    12381  12423.0   
7                   ARF               Coma          No    11453  11487.0   
8     MOSF w/Malignancy                NaN         Yes    12426  12437.0   
9                   ARF                NaN         Yes    11381  11400.0   
10                  ARF                NaN          No    11705  11724.0   
11                  ARF               

In [14]:
print("2. The most important variables we use below are death (patient death recorded/not recorded) and swang1 (rhc performed/not performed). How are these variables coded?")
print("Swang1 Variable")
print(df1['swang1'])
print("Death Variable")
print(df1['death'])
print("They are coded in string")


2. The most important variables we use below are death (patient death recorded/not recorded) and swang1 (rhc performed/not performed). How are these variables coded?
Swang1 Variable
0       No RHC
1          RHC
2          RHC
3       No RHC
4          RHC
5       No RHC
6       No RHC
7       No RHC
8       No RHC
9          RHC
10      No RHC
11         RHC
12         RHC
13      No RHC
14      No RHC
15      No RHC
16         RHC
17         RHC
18      No RHC
19      No RHC
20         RHC
21         RHC
22         RHC
23      No RHC
24      No RHC
25         RHC
26      No RHC
27      No RHC
28      No RHC
29      No RHC
         ...  
5705    No RHC
5706    No RHC
5707    No RHC
5708    No RHC
5709       RHC
5710       RHC
5711       RHC
5712    No RHC
5713       RHC
5714       RHC
5715    No RHC
5716    No RHC
5717    No RHC
5718    No RHC
5719       RHC
5720    No RHC
5721       RHC
5722       RHC
5723    No RHC
5724    No RHC
5725       RHC
5726    No RHC
5727    No RHC
5728    

## 2 Cross-Sectional Estimator

Estimate, using linear regression, how is the RHC related to mortality.

We start easy: let's just find the relationship between recorded death
and rhc.  The important variables are

* _death_: patient death recorded/not recorded
* _swang1_: rhc performed/not performed

This is called "cross-sectional estimator", we just compare
cross-section of individuals who received and did not receive RHC.

Obviously, this is a very crude measure because the hospitals track
patients over different time periods, and if contact is lost early,
the death may not be recorded.  Another obvious problem is that the
patients have very different medical conditions, a factor that
most likely plays a role in the decision whether to perform RHC.


Comment your results.

RHC seems to be associated with 5% larger probability of death.  The
association is highly significant (t=3.9)

In [28]:
# iterates through the rows of the matrix to change No to 0 and Yes to 1 in swang1 and death columns
for i in range(0, len(df1.head(10).index)):
    # changes the value of the swang1 columns
    if (df1['swang1'][i] == 'No RHC'):
        df1['swang1'][i] = 0
    elif (df1['swang1'][i] == 'RHC'):
        df1['swang1'][i] = 1
    # changes the value of the death columns
    elif (df1['death'][i] == 'Yes'):
        df1['death'][i] = 1
    elif (df1['death'][i] == 'No'):
        df1['death'][i] = 0
        
print(df1.head(10))
        

                cat1           cat2          ca  sadmdte  dschdte   dthdte  \
0               COPD            NaN         Yes    11142  11151.0      NaN   
1      MOSF w/Sepsis            NaN          No    11799  11844.0  11844.0   
2  MOSF w/Malignancy  MOSF w/Sepsis         Yes    12083  12143.0      NaN   
3                ARF            NaN          No    11146  11183.0  11183.0   
4      MOSF w/Sepsis            NaN          No    12035  12037.0  12037.0   
5               COPD            NaN          No    12389  12396.0      NaN   
6  MOSF w/Malignancy            NaN  Metastatic    12381  12423.0      NaN   
7                ARF           Coma          No    11453  11487.0  11491.0   
8  MOSF w/Malignancy            NaN         Yes    12426  12437.0      NaN   
9                ARF            NaN         Yes    11381  11400.0      NaN   

   lstctdte death  cardiohx  chfhx  ...   meta  hema  seps  trauma  ortho  \
0     11382     0         0      0  ...     No    No    No      

/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [36]:
df2 = df1.head(10)
x = df2['death']
y = df2['swang1']
print(y)

#slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
#fig, ax = plt.subplots(figsize=(12,8))  
#df2.plot(x= 'death', y='swang1', kind = 'scatter')  
#ax.scatter(df2[0], df2[1], label='black')  
#plt.xticks(())
#plt.yticks(())

#plt.show()

0    0
1    1
2    1
3    0
4    1
5    0
6    0
7    0
8    0
9    1
Name: swang1, dtype: object


## 3 Address some of the issues

Now let's try to address some of the issues with the estimator above.
The dataset includes many other patient descriptors.


### 3.1 Let's include 'age' and 'sex'.
How are these coded?

Age is coded in double and sex is coded in string.

What do you find?

We see age is numeric and sex is 'Female'/'Male'.

In [19]:
print("Age Variable")
print(df1['age'])
print("Sex Variable")
print(df1['sex'])
print("Age is coded in double")
print("Sex is coded in string")
print("")




Age Variable
0       70.250977
1       78.178955
2       46.091980
3       75.331970
4       67.909973
5       86.077942
6       54.967987
7       43.638977
8       18.041992
9       48.423981
10      34.441986
11      68.347961
12      74.709961
13      42.236999
14      81.970947
15      78.304993
16      88.421997
17      69.001953
18      41.428986
19      67.503967
20      50.589996
21      62.688995
22      42.052978
23      23.112991
24      62.107971
25      39.829987
26      47.755981
27      36.700989
28      71.200989
29      95.536987
          ...    
5705    38.942993
5706    68.949951
5707    59.966980
5708    77.368957
5709    68.750000
5710    71.123962
5711    75.635986
5712    23.079987
5713    68.407959
5714    88.779968
5715    61.434998
5716    72.506958
5717    62.638977
5718    46.890991
5719    42.904999
5720    72.292969
5721    74.896973
5722    69.926941
5723    63.772980
5724    63.011993
5725    79.216980
5726    72.035950
5727    61.346985
5728    62.1329

### 3.2 Include age and sex in the regression

Now allow the death to depend on gender and age, on top of that it may
depend on rhc.  Note that it may not just depend on age in a linear
fashion but in a much more complex way, so include not just $age$ but
also $age^2$ and $age^3$ as explanatory variables.

Comment your results

Seems like the association between treatment and rhc is not much
affect, if anything, it got larger.

## 4 Design matrices of the models

Each linear model (and many other models) have associated _design
matrices_.  Design matrix is the matrix of all your explanatory
variables (all x-s) in the final numeric matrix form.  This includes
* adding a constant column
* converting categorical/non-numeric variables into suitable numeric
variables

You next task is to create/extract the design matrices of both of the
models above, investigate their numeric properties (condition
numbers), and solve the linear regression problem in matrix form.

We did not have time in the class to talk about it, but there is a
closed-form solution for the linear regression problem:  beta =
$(X'X)^{-1} X'y$.  Compute this solution and compare with the regression
package output above.


### 4.1 First model

#### 4.1.1 create the design matrix of it, X.

Depending on the way you solved your problem, you may already have
created it.  Depending on the way you solved the problem above, you
may be able to extract it from the existing model.  You may also redo it
manually here.  Remember:
* include the constant term!
* design matrix must be a _matrix_, not data frame or something else.

#### 4.1.2 Compute the condition number of X`X.

You may choose whatever definition you like, but please report what
are you using.

#### 4.1.3 Compute your regression coefficients using the formula above.

Note: you also need your outcome variable $y$ in numeric matrix
form. 

#### 4.1.4 Compare your coefficients here with the OLS results above.

### 4.2 Second Model

Repeat the previous steps 1..4 for the second model, involving $age$,
$age^2$, and $age^3$.

Compare the results with the regression output.  Comment the condition
numbers you got.

The coefficients are the same.  One can see that the condition number
is enormous, so there may be numeric problems.
